In [1]:
import numpy as np
import pandas as pd

In [2]:
instances = pd.read_csv('./kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv')
input = pd.read_csv('./kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv')

input = input[input['player_to_predict'] == True]
radians = np.deg2rad(input['dir'])
input['vx'] = input['s'] * np.sin(radians)
input['vy'] = input['s'] * np.cos(radians)
input = input[['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'vx', 'vy']]

inputs = {
    f"{gid}_{pid}_{nid}": g
    for (gid, pid, nid), g in input.groupby(['game_id', 'play_id', 'nfl_id'])
}

In [3]:
submission = instances.copy(deep=True)

def compute_pos(row):
    factor = .1
    gid, pid, nid, fid = row['game_id'], row['play_id'], row['nfl_id'], row['frame_id']

    ipt = inputs[f"{gid}_{pid}_{nid}"]
    last_row = ipt.iloc[-1]
    pred_x = last_row['x'] + (fid * last_row['vx'] * factor)
    pred_y = last_row['y'] + (fid * last_row['vy'] * factor)
    full_id = f"{gid}_{pid}_{nid}_{fid}"

    return pd.Series([full_id, pred_x, pred_y])

submission[['id', 'x', 'y']] = submission.apply(compute_pos, axis=1)


submission = submission[['id', 'x', 'y']]
submission.head(10)

submission.to_csv('submission.csv', index=False)